In [15]:
### Predict Messages
from langchain.chat_models import ChatOpenAI
from langchain.schema  import HumanMessage, AIMessage, SystemMessage
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]) ## call Chat large language models API.

messages  = [
    SystemMessage(content='you are a geography expert. and you only reply in korean.'),
    AIMessage(content='무엇이 궁금하신가요? 저는 지리학 박사입니다!'),
    HumanMessage(content='한국과 일본의 지리적 차이에 대해 알려줘.')
]

# Pass a message sequence to the model and return a message prediction.
chat.predict_messages(messages)

한국과 일본은 지리적으로 매우 다릅니다. 한국은 한반도에 위치하고 있으며, 주요 산맥은 백두산이 있습니다. 한국은 바다로 둘러싸여 있어 해안선이 길고, 섬도 많이 있습니다. 반면에 일본은 일본 제도로 이루어져 있으며, 홋카이도, 본주, 신주쿠, 사이쿠 등 여러 섬으로 구성되어 있습니다. 또한, 일본은 화산 활동이 활발하여 다양한 화산이 있습니다. 이러한 지리적 차이로 인해 한국과 일본은 각자 독특한 자연환경과 문화를 갖고 있습니다.

AIMessageChunk(content='한국과 일본은 지리적으로 매우 다릅니다. 한국은 한반도에 위치하고 있으며, 주요 산맥은 백두산이 있습니다. 한국은 바다로 둘러싸여 있어 해안선이 길고, 섬도 많이 있습니다. 반면에 일본은 일본 제도로 이루어져 있으며, 홋카이도, 본주, 신주쿠, 사이쿠 등 여러 섬으로 구성되어 있습니다. 또한, 일본은 화산 활동이 활발하여 다양한 화산이 있습니다. 이러한 지리적 차이로 인해 한국과 일본은 각자 독특한 자연환경과 문화를 갖고 있습니다.')

In [16]:
### Prompt Templates
from langchain.prompts import PromptTemplate, ChatPromptTemplate 

# Load a prompt template from a template.
template = PromptTemplate.from_template('{country_a}과 {country_b}의 지리적 차이에 대해 알려줘.')

prompt = template.format(
country_a='korea',
country_b='japan')

# Pass a single string input to the model and return a string prediction.
chat.predict(prompt) 

# Create a chat prompt template from a variety of message format그냥,, s.
template = ChatPromptTemplate.from_messages ([
    ('system', 'you are a geography expert. and you only reply in {language}.'),
    ('ai', '무엇이 궁금하신가요? 저는 {name}입니다!'),
    ('human', '{country_a}과 {country_b}의 지리적 차이에 대해 알려줘.')
])

prompt = template.format_messages(language="korean", name='배다롱', country_a='한국', country_b='일본')

# Pass a message sequence to the model and return a message prediction.
chat.predict_messages(prompt) 

한국과 일본은 지리적으로 많은 차이가 있습니다. 

먼저, 한국은 한반도의 남쪽에 위치하고 있으며, 북쪽으로는 북한과 중국이 인접해 있습니다. 한국은 산이 많고, 해안선이 길며, 기후는 일반적으로 봄, 여름, 가을, 겨울의 네 계절로 나뉩니다. 또한, 한국은 지진이 발생할 수 있는 지역으로 알려져 있습니다.

반면에, 일본은 일본 제도로 이루어진 섬나라로, 주요 섬으로는 홋카이도, 혼슈, 신주쿠, 규슈 등이 있습니다. 일본은 화산 활동이 활발한 지역으로, 많은 화산이 분포하고 있습니다. 또한, 일본은 기후가 다양하며, 북부 지방은 추운 겨울을 경험하고, 남부 지방은 따뜻한 기후를 누릴 수 있습니다.

이처럼, 한국과 일본은 지리적으로 다양한 특징을 가지고 있으며, 이러한 차이로 인해 두 나라의 문화, 역사, 자연환경 등이 다르게 발전해 왔습니다.한국과 일본은 두 나라 모두 동아시아 지역에 위치하고 있습니다. 한국은 한반도에 위치하고 있으며, 북쪽으로는 북한과 접하고 남쪽으로는 한국 해협을 사이에 두고 일본과 맞닿아 있습니다. 일본은 일본 제도로 이루어져 있으며, 주요 섬으로는 홋카이도, 혼슈, 신주쿠, 규슈 등이 있습니다. 두 나라는 지형적으로도 다양한 산악 지형과 해안선을 가지고 있으며, 기후적으로도 차이가 있습니다.

AIMessageChunk(content='한국과 일본은 두 나라 모두 동아시아 지역에 위치하고 있습니다. 한국은 한반도에 위치하고 있으며, 북쪽으로는 북한과 접하고 남쪽으로는 한국 해협을 사이에 두고 일본과 맞닿아 있습니다. 일본은 일본 제도로 이루어져 있으며, 주요 섬으로는 홋카이도, 혼슈, 신주쿠, 규슈 등이 있습니다. 두 나라는 지형적으로도 다양한 산악 지형과 해안선을 가지고 있으며, 기후적으로도 차이가 있습니다.')

In [17]:
from langchain.schema import BaseOutputParser

# Output parsers help structure language model responses.
# My personal output parser
class CommaOutputParser(BaseOutputParser):
    # Parse a single string model output into some structure.
    def parse(self,text): 
        items= text.strip().split(',')
        return list(map(str.strip, items))
    
    
p = CommaOutputParser()
p.parse("hello, how, are, you?")    
# ['hello', 'how', 'are', 'you?']

['hello', 'how', 'are', 'you?']

In [18]:

template = ChatPromptTemplate.from_messages([
    ("system", "you are a list generating machine. everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. do not reply with anything else."),
    ("human", "{question}"),
])

# general way
# prompt = template.format_messages(max_items=10, question = 'what are the colors?')
# results = chat.predict_messages(prompt)
# p = CommaOutputParser()
# p.parse(results.content)

# LangChain Expression Language, or LCEL, is a declarative way to easily compose chains together. 
# RunnableSequence을 사용하여 여러 Runnable을 조합하고 체인을 구성할 수 있습니다. 
chain = template | chat | CommaOutputParser()
chain.invoke({"max_items":5, "question": "flowers"})

rose, daisy, tulip, sunflower, lily

['rose', 'daisy', 'tulip', 'sunflower', 'lily']

In [19]:
# Chaining Chains
chef_template = ChatPromptTemplate.from_messages([
    ("system", "you are a world-class international chef. you create easy to follow recipies for any typeof cuisins with easy to find ingredients."),
    ("human", "I want to cook {cuision} food."),
])

chef_chain = chef_template | chat 

In [20]:
veg_chef_template = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_template | chat

In [21]:
final_chain = {"recipe": chef_chain} | veg_chain
final_chain.invoke({'cuision':'indian'})

Great choice! Here's a simple recipe for Chicken Tikka Masala, a popular Indian dish:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 teaspoons ground cumin
- 2 teaspoons paprika
- 1 teaspoon ground cinnamon
- 1 teaspoon ground turmeric
- 1 teaspoon ground coriander
- 1 teaspoon cayenne pepper (adjust to taste)
- Salt and pepper to taste
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon grated ginger
- 1 can (14 oz) tomato sauce
- 1 cup heavy cream
- Fresh cilantro, chopped (for garnish)

Instructions:
1. In a bowl, mix together the yogurt, lemon juice, cumin, paprika, cinnamon, turmeric, coriander, cayenne pepper, salt, and pepper. Add the chicken pieces and coat them well with the marinade. Cover and refrigerate for at least 1 hour, or overnight for best results.

2. Preheat the oven to 400°F (200°C). Place the marinated chicken pieces on a baking sh

AIMessageChunk(content="To make this Chicken Tikka Masala recipe vegetarian, we can replace the chicken with a suitable alternative. One popular option is to use paneer, a type of Indian cottage cheese that is firm and holds its shape well when cooked. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb paneer, cut into bite-sized pieces\n- 1 cup plain yogurt\n- 2 tablespoons lemon juice\n- 2 teaspoons ground cumin\n- 2 teaspoons paprika\n- 1 teaspoon ground cinnamon\n- 1 teaspoon ground turmeric\n- 1 teaspoon ground coriander\n- 1 teaspoon cayenne pepper (adjust to taste)\n- Salt and pepper to taste\n- 2 tablespoons vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1 tablespoon grated ginger\n- 1 can (14 oz) tomato sauce\n- 1 cup heavy cream\n- Fresh cilantro, chopped (for garnish)\n\nInstructions:\n1. Follow the same marinating process as the original recipe, but substitute the chicken with paneer cubes. Coat the paneer well with the marinade and refrige